# Outline
- [ 1 - Importing the required Library and Packages ](#1)
- [ 2 - Various helper function code block](#2)
  - [ 2.1 Converting Original image into Gray Image](#2.1)
  - [ 2.2 Hough Circle Detection](#2.2)
  - [ 2.3 Canny Edge using Dilate and Erode Method](#2.3)
  - [ 2.4 Flood Fill Function to fill any region with color](#2.4)
  - [ 2.5 Contour Detection Function](#2.5)
  - [ 2.6 Finding the Two Largest Contours](#2.6)
  - [ 2.7 Centroid function to get the centroid of the contours](#2.7)
- [ 3 - Detecting the Firing Pin](#3)
  - [ 3.1 Finding Tip of the Firing Pin](#3.1)
  - [ 3.2 Detecting the direction of Firing Pin](#3.2)
- [ 4 - Aperture Shear Functions](#4)
  - [ 4.1 Aperture Shear Direction](#4.1)
  - [ 4.2 Draw the Aperture Shear Arc](#4.2)
- [ 5 Image Masking](#5)
  - [ 5.1 ImageMask Left](#5.1)
  - [ 5.2 ImageMask Right](#5.2)
  - [ 5.3 ImageMask Left](#5.3)
- [ 6 Main Code](#6)


<a name="1"></a>
## 1 - Importing the required Library and Packages 

First, let's run the cell below to import all the packages that you will need during this assignment.
- [numpy](www.numpy.org) is the fundamental package for scientific computing with Python.
- [matplotlib](http://matplotlib.org) is a famous library to plot graphs in Python.
- [opencv (cv2)](https://docs.opencv.org/4.x/) Library for working with image and carrying out various image/video processing algorithms
- [imutils](https://pypi.org/project/imutils/)
    - [imutils latest docs](https://imutils.readthedocs.io/en/latest/) A series of convenience functions to make basic image processing functions such as translation, rotation, resizing, skeletonization, displaying Matplotlib images, sorting contours, detecting edges

In [1]:
import cv2
import numpy as np
import math
import imutils

<a name="2"></a>
## 2 - Various helper function code block

<a name="2.1"></a>
### 2.1 - Converting Original image into Gray Image

In [2]:
# Gray image function
def gray_img(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Converting the img to Gray using cvtColor method
    gray = cv2.medianBlur(gray, 5)
    return gray # returning the gray image

<a name="2.2"></a>
### 2.2 - Hough Circle Detection

#### This Function is used to find the circular region of the bullet cartridge in the given image

In [3]:
# This function
def hough_circles(gray):
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 800, param1=80, param2=50, minRadius=250, maxRadius=0)
    
    detected_circles = np.uint16(np.around(circles))
    for (x, y,r) in detected_circles [0, :]:
        cv2.circle(input_img, (x, y), r, (0, 255, 0), 3)
        cv2.circle(input_img, (x, y), 2, (0, 255, 255), 10)
    return input_img, x, y, r

<a name="2.3"></a>
### 2.3 - Canny Edge using Dilate and Erode Method.

In [4]:
# Dilate and Erode on gray image
def dilate_erode_img(gray):
    edge = cv2.Canny(img, 150, 100)

    # Taking a matrix of size 3 as the kernel 
    kernel = np.ones((3, 3), np.uint8)
    edge = cv2.dilate(edge, kernel, iterations=2)
    
    kernel = np.ones((4, 4), np.uint8)
    edge = cv2.erode(edge, kernel, iterations=3)
    return edge

<a name="2.4"></a>
### 2.4 - Flood Fill Function to fill any region with color

In [5]:
# Flood fill the image with color to highlight the pin impression on bullet catridge
def flood_fill(img, floodfill_color, x, y, mask = None):
    seed_point = x,y
    # print(seed_point)
    cv2.floodFill(img, mask, seed_point, floodfill_color)

<a name="2.5"></a>
### 2.5 Contour Detection Function

In [6]:
# Finding contours
# apply binary thresholding
def contour_img(edge, input_img): #sobel - 3rd parameter
    ret, thresh = cv2.threshold(edge, 150, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(image=thresh, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(image=input_img, contours=contours, contourIdx=-1, color=(0, 0, 255), thickness=2, lineType=cv2.LINE_AA)

<a name="2.6"></a>
### 2.6 Finding the Two Largest Contours (i.e the contour within Hough Circle which will be Breach face Impression and Pin Impression 

In [7]:
def draw_largest_two_contours(canny_edge, img_copy):
    # apply binary thresholding
    second_img = img_copy.copy()
    third_img = img_copy.copy()
    ret, thresh = cv2.threshold(canny_edge, 150, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(image=thresh, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_SIMPLE)

# Select the second-largest contour
    if len(contours) >= 2:
        two_largest_contours = sorted(contours, key=cv2.contourArea, reverse=True)[:2]
        second_largest_contour = two_largest_contours[1]
        
        # Draw only the two largest contours
        cv2.drawContours(image = img_copy, contours = two_largest_contours, contourIdx =-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
        cv2.drawContours(image = second_img, contours = [second_largest_contour], contourIdx =-1, color=(0, 255, 0), thickness=4, lineType=cv2.LINE_AA)

    return two_largest_contours, second_largest_contour

<a name= "2.7"></a>
### 2.7 - Centroid function to get the centroid of the contours

In [8]:
def centroid_two_largest_contours(two_largest_contours, input_img):

# Calculate centroids for the two largest contours
    centroid_pairs = []
    for cnt in two_largest_contours:
        moments = cv2.moments(cnt)
        # print(moments)
        if moments['m00'] != 0:
            cx = int(moments['m10'] / moments['m00'])
            cy = int(moments['m01'] / moments['m00'])
            # print('Plain cx & cy', cx, cy)
            centroid_pairs.append((cx,cy))
            # Draw a circle at the centroid
            cv2.circle(input_img, (cx, cy), 5, (0, 0, 255), -1)
            # print(centroid_pairs)
    return centroid_pairs

<a name="3"></a>
## 3 - Pin Direction but using the tip of the pin with help of contour_points of 2nd largest contours

<a name="3.1"></a>
### 3.1 -Finding Tip of the Firing pin through the Pin tip(Top) points and centroid

In [9]:
def get_tip_point(contour_points, centroid):
    distances = np.linalg.norm(contour_points - centroid[1], axis=1)
    tip_point_index = np.argmax(distances) # This give the maximum distance between tip point (i.e Pin Point and Centroid of the pin Contours)
    tip_point = contour_points[tip_point_index]
    return tip_point

<a name="3.2"></a>
### 3.2 Detecting the direction of Firing Pin
#### The `Pin_Tip_Direction` Function

#### This function will dynamically find the pin tip and draw a line to it and using that line we will be drawing a perpendicular line to find the region of the following

- `Firing Pin Drag`
- `Firing Pin Impression`
- `Direction of the Firing Pin Drag`

In [10]:
def pin_tip_direction(centroid_pairs, tip_point, image):
    start_point = centroid_pairs[1]
    end_point = tip_point

# Pin Direction ArrowHead Logic
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++    
    # Calculate a vector along the arrow direction
    arrow_direction = np.subtract(end_point, start_point)

    # Adaptive arrow length based on the distance between start and end points
    arrow_length = int(2 * np.linalg.norm(arrow_direction))

    # Limit the arrow length to the distance between start and end points
    arrow_length = min(arrow_length, int(np.linalg.norm(arrow_direction)))
    # print('min arrow', arrow_length)

    # Calculate the endpoint based on the limited arrow length
    arrow_scale_factor = 0.8
    scaled_arrow_length = int(arrow_scale_factor * arrow_length)
 
    # Calculate the adjusted endpoint based on the scaled arrow length
    adjusted_end_point = (start_point[0] + int(arrow_direction[0] * arrow_scale_factor),
                          start_point[1] + int(arrow_direction[1] * arrow_scale_factor))

    # Red color in BGR  
    color = (168, 52, 34) 

    # cv2.circle(image, start_point, 10, color, 3) # Circle at starting point which is centroid
    # cv2.circle(image, adjusted_end_point, 10, color, 3) # Circle at starting point which is centroid
    image_copy = cv2.arrowedLine(image, start_point, tuple(map(int, adjusted_end_point)), color, thickness = 4)

#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


# Perpendicular line logic 
# ---------------------------------------------------------------------------------
    # Calculate the slope of the arrow line
    slope = (end_point[1] - start_point[1]) / (end_point[0] - start_point[0])
    
    # Calculate the direction vector for the perpendicular line
    perpendicular_direction = np.array([-1, 1 / slope])
    
    # Normalize the direction vector
    perpendicular_direction /= np.linalg.norm(perpendicular_direction)
    
    # Choose a length for the perpendicular line
    perpendicular_line_length = 80
    
    
    # Calculate the end points of the perpendicular line in both directions
    perpendicular_end_point_1 = (int(start_point[0] + perpendicular_direction[0] * perpendicular_line_length),
                                 int(start_point[1] + perpendicular_direction[1] * perpendicular_line_length))
    
    perpendicular_end_point_2 = (int(start_point[0] - perpendicular_direction[0] * perpendicular_line_length),
                                 int(start_point[1] - perpendicular_direction[1] * perpendicular_line_length))
    # Draw the perpendicular line

    # Draw the perpendicular line in both directions
    cv2.line(image_copy, start_point, perpendicular_end_point_1, color, thickness=4)
    cv2.line(image_copy, start_point, perpendicular_end_point_2, color, thickness=4)
# ---------------------------------------------------------------------------------

    # print('Arrow Direction', arrow_direction)
    return image_copy, arrow_direction

<a name="4"></a>
## 4 - Function to find Aperture Shear and will draw the arc on it

<a name="4.1"></a>
### 4.1 - ApertureShear Direction Function will help us to identify the point of aperture shear

In [11]:
def apertureshear_direction_for_arc(centroid_pairs, tip_point, image):
    end_point = centroid_pairs[1]
    start_point = tip_point

     # Calculate a vector along the arrow direction
    arrow_direction = np.subtract(end_point, start_point)
    
    # Extend the line by a certain factor (e.g., 1.5 times the length of the arrow)
    aperture_shear_point = (end_point[0] + int(1.15 * arrow_direction[0]), end_point[1] + int(1.15 * arrow_direction[1]))

    color = (0, 255, 255)
    cv2.circle(image, aperture_shear_point, 12, color, 3)
        
    return aperture_shear_point

<a name="4.2"></a>
### 4.2 Draw the Aperture Shear Arc  `draw_apertureshear_arc` will draw and arc for it

In [12]:
# Function 3: Draw an arc passing through a point (arc_start_point)
def draw_apertureshear_arc(image, centroid_pairs, arc_start_point):

    # Calculate the vector from centroid to arc_start_point
    vector_to_start_point = np.subtract(arc_start_point, centroid_pairs[1])

    # Calculate the radius as half the distance between centroid and arc_start_point
    radius = int(np.linalg.norm(vector_to_start_point))

    # Calculate the start and end angles for the arc (assuming a full circle)
    start_angle = 0
    end_angle = 30

    center = centroid_pairs[1]
    
    # Red color in BGR
    color = (0, 128, 128)

    # Draw the arc passing through the arc_start_point
    image_with_arc = image.copy()
    
     # Draw the arc passing through the arc_start_point
    cv2.ellipse(image_with_arc, center, (radius, radius), 0, start_angle, end_angle, color, thickness=6)

    return image_with_arc

<a name = "5"></a>
## 5 - Image Masking Function to use to identify the region to floodfill with different color

### Function Image Mask will create a mask image of the given input image

Return the following parameter
- `image_with_contours`
- `image_mask`

In [13]:
def image_mask(image_with_contours, contours):
    
    # # Create a binary mask based on pin contours
    # mask = np.zeros_like(image_with_contours, dtype=np.uint8)
    # cv2.drawContours(mask, contours, -1, (255, 255, 255), thickness=cv2.FILLED)

    # Convert image with contours to BGR color image
    image_with_contours = cv2.cvtColor(image_with_contours, cv2.COLOR_GRAY2BGR)
    
    # Create a binary mask based on pin contours
    mask = np.zeros_like(image_with_contours, dtype=np.uint8)
    cv2.drawContours(mask, contours, -1, (255, 255, 255), thickness=cv2.FILLED)

    return image_with_contours, mask

<a name="5.1"></a>
### 5.1 - Function imagemask_left return the left region (Firing Pin Drag region mask)

In [14]:
def imagemask_left(mask, centroid_pairs):
    # mask_copy1 = mask.copy()
    # flood fill
    color = 251, 206, 135
    color_arr = np.array(color)
    flood_fill(mask, color, centroid_pairs[1][0] - 25, 
               centroid_pairs[1][1] -25)
    mask_left = cv2.inRange(mask, color_arr, color_arr)
    
    # Mask on image
    mask_left = cv2.threshold(mask_left, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    return mask_left

<a name="5.2"></a>
### 5.2 - Function imagemask_right return the right region (Firing Pin Impression region mask)

In [15]:
def imagemask_right(mask, centroid_pairs):
    # flood fill
    # mask_copy2 = mask.copy()
    color = 128,0,128
    color_arr = np.array(color)
    flood_fill(mask, color, centroid_pairs[1][0] + 25, 
               centroid_pairs[1][1] + 25)
    mask_right = cv2.inRange(mask, color_arr, color_arr)

    # Mask on image
    mask_right = cv2.threshold(mask_right, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    return mask_right

<a name="5.3"></a>
### 5.3 Function imagemask_outer return the outer region (Breach Face Impression region mask)

In [16]:
def imagemask_outer(image, mask, pin_tip_point):
    # Floodfill mask region
    # image_copy = image.copy()
    # mask on outer region of contours
    color = 52, 66, 227
    color_arr = np.array(color)
    flood_fill(mask, color, tip_point[0] - 15, 
               tip_point[1])
    mask_outer = cv2.inRange(mask, color_arr, color_arr)

    # Mask on image
    mask_outer = cv2.threshold(mask_outer, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    return mask_outer


<a name = "6"></a>
# 6 - Main Code File

In [22]:
# Reading the image file
img = cv2.imread('bullet_case2.jpg')
img = imutils.resize(img, width=540)  # .resize(img, (540, 540), interpolation = cv2.INTER_LINEAR)
input_img = img.copy()  # Creating the copy of img in order to keep the original resized img same
image_copy = img.copy()

# Create Gray image of input image
gray = gray_img(input_img)
contour_gray = gray.copy() # Creating a copy of gray image

# Call the hough_circle to detect the circls of the bullets
circle_detected_img, x, y, r = hough_circles(gray)
breach_impression_hough_circles = circle_detected_img.copy()

# calls the dilate and erode function which use canny edge detection
canny_edge_img = dilate_erode_img(gray)
canny_edge = canny_edge_img.copy()
copy_canny_edge = canny_edge_img.copy() # Making Copy of Canny Edge Img

# flood fill the pin impression on the fired bullet cartrigde
color = 106,50,159 
flood_fill(canny_edge_img, color, x, y)

# Calls the function to return two largest contours which is out circles detect and the pin impression contours
two_largest_contours, second_largest_contour = draw_largest_two_contours(canny_edge_img, input_img)

both_contour_image = input_img.copy()

# Draw only the two largest contours on gray_image
cv2.drawContours(image = contour_gray, contours = two_largest_contours, contourIdx =-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)

centroid_pairs = centroid_two_largest_contours(two_largest_contours, input_img)
# print('Pin Contours point', centroid_pairs[1])
cv2.circle(both_contour_image, centroid_pairs[1], 8, (0,0,255), -1)


# Direction of the Pin Code block
# ========================================================================================================
# Squeeze the second_largest_contour
contours_points = np.squeeze(second_largest_contour)

# pin tip points
tip_point = get_tip_point(contours_points, centroid_pairs)
# cv2.circle(pin_drag_onImg, (tip_point), 12, (255,0,255), 4)

# Draw direction of pin on canny edge
pin_drag_img, direction = pin_tip_direction(centroid_pairs, tip_point, copy_canny_edge)

# Draw direction of pin on input Img
pin_drag_onImg, direction = pin_tip_direction(centroid_pairs, tip_point, input_img) 
# ========================================================================================================
# --------------------------------------------------------------------------------------------------------
pin_tip_img = img.copy()
pin_tip_point_Img, direction = pin_tip_direction(centroid_pairs, tip_point, pin_tip_img)

result_image_color, mask = image_mask(contour_gray, two_largest_contours)
mask_img, direction = pin_tip_direction(centroid_pairs, tip_point, mask) 
# --------------------------------------------------------------------------------------------------------


#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Masking 
mask_left = imagemask_left(mask_img, centroid_pairs)
mask_right = imagemask_right(mask_img, centroid_pairs)

pin_drag_onImg[mask_right==255] = (128,0,128)
pin_drag_onImg[mask_left==255] = (251, 206, 135)

result_image_color[mask_right==255] = (128,0,128)
result_image_color[mask_left==255] = (251, 206, 135)

mask_outer = imagemask_outer(contour_gray, mask, tip_point)

pin_drag_onImg[mask_outer==255] = (52, 66, 227)
result_image_color[mask_outer==255] = (52, 66, 227)

#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


arc_start_point = apertureshear_direction_for_arc(centroid_pairs, tip_point, image_copy)
pin_drag_onImg = draw_apertureshear_arc(pin_drag_onImg, centroid_pairs, arc_start_point)

arc_img = draw_apertureshear_arc(result_image_color, centroid_pairs, arc_start_point)
arc_img, direction = pin_tip_direction(centroid_pairs, tip_point, arc_img)

cv2.imshow('Original Image', image_copy)
cv2.imshow('Original Image', gray)
cv2.imshow('Canny Edge', canny_edge)
cv2.imshow('Hough Circle frame', breach_impression_hough_circles)
cv2.imshow('Pin Drag frame', pin_drag_img)
cv2.imshow('Pin directn on O/P', pin_drag_onImg) 
cv2.imshow('Pin tip direction on O/P', pin_tip_point_Img)
cv2.imshow('Both Contours Img', both_contour_image)
cv2.imshow('Result Image (Color with Contours)', result_image_color)
cv2.imshow('Result Image (gray with Contours)', contour_gray)
cv2.imshow('Mask Img', mask_img)
cv2.imshow('Mask Left', mask_left)
cv2.imshow('Mask Right', mask_right)
cv2.imshow('Mask Outer', mask_outer)
cv2.imshow('arc Outer', arc_img)
cv2.imshow('Centrid Point', both_contour_image)

cv2.waitKey(0)
cv2.destroyAllWindows()